In [3]:
%pip install --upgrade --quiet sqlalchemy langchain langchain-community langchain-openai pandas python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

In [7]:
df = pd.read_csv("110225_lol_champion_data.csv")
df.head(5)

,Unnamed: 0,id,apiname,title,difficulty,herotype,alttype,resource,stats,rangetype,...,be,rp,skill_i,skill_q,skill_w,skill_e,skill_r,skills,fullname,nickname
0,Aatrox,266.0,Aatrox,the Darkin Blade,2,Fighter,Tank,Blood Well,"{'hp_base': 650, 'hp_lvl': 114, 'mp_base': 0, ...",Melee,...,4800,880,{1: 'Deathbringer Stance'},"{1: 'The Darkin Blade', 2: 'The Darkin Blade 3'}",{1: 'Infernal Chains'},{1: 'Umbral Dash'},{1: 'World Ender'},"{1: 'Deathbringer Stance', 2: 'The Darkin Blad...",NaN,NaN
1,Ahri,103.0,Ahri,the Nine-Tailed Fox,2,Mage,Assassin,Mana,"{'hp_base': 590, 'hp_lvl': 104, 'mp_base': 418...",Ranged,...,3150,790,{1: 'Essence Theft'},{1: 'Orb of Deception'},{1: 'Fox-Fire'},{1: 'Charm'},{1: 'Spirit Rush'},"{1: 'Essence Theft', 2: 'Orb of Deception', 3:...",NaN,NaN
2,Akali,84.0,Akali,the Rogue Assassin,2,Assassin,NaN,Energy,"{'hp_base': 600, 'hp_lvl': 119, 'mp_base': 200...",Melee,...,3150,790,"{1: ""Assassin's Mark""}",{1: 'Five Point Strike'},{1: 'Twilight Shroud'},{1: 'Shuriken Flip'},{1: 'Perfect Execution'},"{1: ""Assassin's Mark"", 2: 'Five Point Strike',...",Akali Jhomen Tethi,NaN
3,Akshan,166.0,Akshan,the Rogue Sentinel,3,Marksman,Assassin,Mana,"{'hp_base': 630, 'hp_lvl': 107, 'mp_base': 350...",Ranged,...,4800,880,{1: 'Dirty Fighting'},{1: 'Avengerang'},{1: 'Going Rogue'},{1: 'Heroic Swing'},{1: 'Comeuppance'},"{1: 'Dirty Fighting', 2: 'Avengerang', 3: 'Goi...",NaN,NaN
4,Alistar,12.0,Alistar,the Minotaur,1,Tank,Support,Mana,"{'hp_base': 685, 'hp_lvl': 120, 'mp_base': 350...",Melee,...,1350,585,{1: 'Triumphant Roar'},{1: 'Pulverize'},{1: 'Headbutt'},{1: 'Trample'},{1: 'Unbreakable Will'},"{1: 'Triumphant Roar', 2: 'Pulverize', 3: 'Hea...",NaN,NaN


In [15]:
#carga csv en base de datos sqlite
import sqlite3
from sqlalchemy import create_engine

#crear un motor de base de datos SQLite con SQLALCHEMY
engine = create_engine("sqlite:///lol_champions.db")

df.to_sql("champions", engine, if_exists="replace", index=False)

172

In [23]:
#importar dotenv para cargar variables de entorno 
from dotenv import load_dotenv
import os

from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
#cargar variables de entorno desde el archivo .env
load_dotenv()
#conectar el agente lLangchain con la base de datos SQL
db = SQLDatabase(engine=engine)
#instanciar el LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'))
#creamos el agente con el executor
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)

In [25]:
#CONSULTAR DATOS
response = agent_executor.invoke({"input": "¿Cuántos campeones hay en total en el dataset?"})
print(response["output"])

Hay un total de 172 campeones en el dataset.


In [27]:
#verificacion cruzada
len(df)

172

In [29]:
response = agent_executor.invoke({"input": "¿Cuál es el campeón con mayor HP base en el juego?"})

print(response["output"])

El campeón con el mayor HP base en el juego es Kled, con un HP base de 810.


In [38]:
#crear agente con verbose=True
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)

response = agent_executor.invoke({"input": "¿Cuáles son los 5 campeones más dificiles de jugar segun su nivel de dificultad y ordenalos?"})
print(response["output"])

Parece que hay un problema con los datos, ya que varios campeones no tienen un nombre completo asociado. Sin embargo, todos los campeones en los resultados tienen un nivel de dificultad de 3, que parece ser el nivel más alto en la base de datos. 

Si necesitas más información o un análisis diferente, por favor házmelo saber.


In [40]:
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)

response = agent_executor.invoke({"input": "¿Cuantos campeones hay por cada rol principal?"})
print(response["output"])

Aquí tienes el número de campeones por cada rol principal:

1. Tirador (Marksman): 21 campeones
2. Vanguardia (Vanguard): 16 campeones
3. Luchador (Diver): 15 campeones
4. Especialista (Specialist): 14 campeones
5. Escaramuzador (Skirmisher): 14 campeones
6. Coloso (Juggernaut): 14 campeones
7. Asesino (Assassin): 13 campeones
8. Explosión (Burst): 12 campeones
9. Mago de batalla (Battlemage): 11 campeones
10. Encantador (Enchanter): 8 campeones
